In [ ]:
import pandas as pd
import numpy as np 
import os
import ast

import plotly.express as px
import plotly.io as pio

pd.options.plotting.backend = "plotly"
pio.templates.default = "plotly_white"

In [ ]:
wd = "/Users/annie.quinn/git/ford-hackathon/personal/annie/AO21"

In [ ]:
# Get filtered audience origin data
# Focus on wave 1 - higher number of responses for deep dive
data = pd.read_csv(os.path.join(wd, "ao_data.csv"), index_col=0)
data = data.loc[data["Quarter"] == 1]
data

In [ ]:
# Get tabled meta data
col_desc_values = pd.read_csv(os.path.join(wd, "ao_col_desc_values.csv"), index_col=0)
col_desc_values.dropna(inplace=True)
col_desc_values = col_desc_values.loc[col_desc_values["col"].isin(data.columns)]
col_desc_values.reset_index(drop=True, inplace=True)
col_desc_values

In [ ]:
cols_constant = ["Market"]
cols_grp = ["Q01_NET", "Q02"]
col_pop = "Core_Projection_Online"
cols_cat_purchase = ["PUR03_2_46", "PUR03_2_53"]
cols_tp_noticed = col_desc_values.loc[col_desc_values["description"].str.contains("Touchpoints noticed | A car", regex=False), "col"].values.tolist()
cols_tp_helped = col_desc_values.loc[col_desc_values["description"].str.contains("Touchpoints helped | A car", regex=False), "col"].values.tolist()
cols_tp = [*cols_tp_noticed, *cols_tp_helped]

cols = [*cols_constant, *cols_grp, col_pop, *cols_cat_purchase, *cols_tp]

df = data[cols]
idx = df[cols_tp].dropna(how="all").index
df = df.loc[idx]

df.isnull().sum().sum() # Should be zero

In [ ]:
df

In [ ]:
# If helped is selected, noticed must also be selected
tp_suffixes = [x[5:] for x in df.columns[df.columns.str.startswith("TP02")].values.tolist()]
tp_suffixes

for suffix in tp_suffixes:
    cols_suffix = df.columns[df.columns.str.endswith(suffix)]
    if len(cols_suffix) != 2: 
        print(suffix)
    else: 
        tp02 = cols_suffix[cols_suffix.str.startswith("TP02")].values[0]
        tp03 = cols_suffix[cols_suffix.str.startswith("TP03")].values[0]
        mask = df[tp03] > df[tp02]
        df.loc[mask, tp02] = 1

In [ ]:
# Remap each column
for col in cols:
    if col != col_pop:
        mapping = ast.literal_eval(col_desc_values.loc[col_desc_values["col"] == col, "values"].values[0])
        df[col] = df[col].replace(mapping)

In [ ]:
# Reshape
df_long = df.melt(id_vars = [*cols_constant, *cols_grp, col_pop, *cols_cat_purchase])

# Add car type
df_long["car_type"] = np.where(df_long["PUR03_2_46"]=="Selected", "new", "second_hand")
df_long.drop(columns=cols_cat_purchase, inplace=True)

# Aggregate data
df_long = (
    df_long
    .groupby([*cols_constant, *cols_grp, "car_type", "variable", "value"])
    [col_pop].sum().reset_index()
    .rename(columns={col_pop:"pop"})
)
df_long

In [ ]:
# Simplify vars
map_col_desc = dict(zip(col_desc_values["col"], col_desc_values["description"]))
df_long["variable_str"] = df_long["variable"].replace(map_col_desc)
df_long[["var1", "var2", "var3"]] = df_long["variable_str"].str.split(" | ", regex=False, expand=True)

map_age = dict(zip(df_long["Q01_NET"].unique().tolist(), ["<35", "<35", ">35", ">35", ">35"]))
df_long["age"] = df_long["Q01_NET"].replace(map_age)

#df_long["age_sex"]
for idx, row in df_long.iterrows():
    df_long.loc[idx, "age_sex"] = row["Q02"] + " " + row["age"]

df_long

In [ ]:
# Tidy and reaggregate
df_long = (
    df_long
    [["Market", "car_type", "variable", "var1", "var3", "age_sex", "value", "pop"]]
    .groupby(["Market", "car_type", "variable", "var1", "var3", "age_sex", "value"])
    ["pop"].sum().reset_index()
)
df_long.to_csv("AO21/AO_data_tidy.csv")
df_long

-----

In [ ]:
# Percent buying NEW cars? # USE THIS FIGURE
(
    df_long
    .groupby(["Market", "car_type"])
    ["pop"].sum().reset_index()
    .assign(pct = lambda d: d["pop"] / d.groupby(["Market"])["pop"].transform("sum") * 100)
    .plot.bar(x = "Market", y = "pct", color = "car_type", barmode = "group", text = "pct")
)

In [ ]:
df_long = df_long.copy().loc[df_long["car_type"] == "new"] # Override, focus on new only

In [ ]:
# WHO is buying the new cars # USE THIS FIGURE
(
    df_long.groupby(["Market", "age_sex"])["pop"].sum().reset_index()
    .assign(pct = lambda d: d["pop"] / d.groupby(["Market"])["pop"].transform("sum") * 100)
    .plot.bar(x = "Market", y = "pct", color = "age_sex", barmode="group", text = "pct")
)

In [ ]:
# Touchpoints, percent, aggregated

In [ ]:
tp_pct = (
    df_long
    .groupby(["Market", "var1", "var3", "value"])
    ["pop"].sum().reset_index()
    .pivot(index = ["Market", "var1", "var3"], columns = "value", values = "pop")
    .reset_index()
    .assign(
        pct = lambda d: d["Selected"] / (d["Selected"] + d["Not selected"]) * 100,
        rank = lambda d: d.groupby(["Market", "var1"])["pct"].rank(ascending=False)
    )
)

In [ ]:
(
    tp_pct
    .plot.bar(x = "var3", y = "pct", facet_col = "Market", facet_row = "var1", text = "rank")
    .update_layout(height=800)
)

In [ ]:
(
    tp_pct
    .query("rank <= 5")
    .plot.bar(x = "var3", y = "pct", facet_col = "Market", facet_row = "var1", text = "rank")
    .update_layout(height=800)
)

In [ ]:
# Translation from noticed to helped
(
    tp_pct
    .pivot(index = ["Market", "var3"], columns = "var1", values = "pct").reset_index()
    .plot.scatter(x = "Touchpoints noticed", y = "Touchpoints helped", color = "var3", facet_col = "Market")
    .update_layout(height=800)
    .update_traces(marker = dict(size=15))
)

In [ ]:
tp_pct.plot(x = "var1", y = "pct", color = "var3", facet_col = "Market", category_orders = {"var1": ["Touchpoints noticed", "Touchpoints helped"]}).update_layout(height = 800)

In [ ]:
# conversions, the ratio of conversions to interactions, i.e. the ratio of noticed to helped
tp_pct

In [ ]:
# conversions, the ratio of conversions to interactions, i.e. the ratio of noticed to helped  # USE THIS FIGURE
((
    tp_pct.pivot(index = ["Market", "var3"], columns = "var1", values = "Selected").reset_index()
    .assign(conversion_rate = lambda d: d["Touchpoints helped"] / d["Touchpoints noticed"] * 100)
).merge(tp_pct.query("var1=='Touchpoints noticed'")[["Market", "var3", "pct", "rank"]])
    .plot.scatter(x = "conversion_rate", y = "pct", color= "var3", facet_col = "Market")
    .update_layout(height=800)
)

In [ ]:
# Touchpoint, percent, age group
tp_pct = (
    df_long
    .groupby(["Market", "age_sex", "var1", "var3", "value"])
    ["pop"].sum().reset_index()
    .pivot(index = ["Market", "var1", "var3", "age_sex"], columns = "value", values = "pop")
    .reset_index()
    .assign(
        pct = lambda d: d["Selected"] / (d["Selected"] + d["Not selected"]) * 100,
        rank = lambda d: d.groupby(["Market", "var1", "age_sex"])["pct"].rank(ascending=False)
    )
)

In [ ]:
(
    tp_pct
    .plot.bar(x = "var3", y = "pct", facet_col = "Market", facet_row = "var1", text = "rank", color = "age_sex", barmode = "group")
    .update_layout(height=800)
)

In [ ]:
(
    tp_pct
    .query("rank <= 5")
    .plot.bar(x = "var3", y = "pct", facet_col = "Market", facet_row = "var1", text = "rank", color = "age_sex", barmode = "group")
    .update_layout(height=800)
)

In [ ]:
(
    tp_pct
    .query("rank <= 3")
    .plot.bar(x = "var3", y = "pct", facet_col = "Market", facet_row = "var1", text = "rank", color = "age_sex", barmode = "group")
    .update_layout(height=800)
)

In [ ]:
# Translation from noticed to helped # USE THIS, HIGHLIGHT RANKS
(
    tp_pct
    .pivot(index = ["Market", "var3", "age_sex"], columns = "var1", values = "pct").reset_index()
    .plot.scatter(x = "Touchpoints noticed", y = "Touchpoints helped", color = "var3", facet_col = "Market", facet_row = "age_sex")
    .update_layout(height=1200)
    .update_traces(marker = dict(size=15))
)